In [75]:
import os 
import json 
import pandas as pd

from datetime import datetime

In [83]:
data = []

In [84]:
dir = "data/music_related/"
ignore_files = ["all_music_tweets.ndjson", ".ipynb_checkpoints"] # ignore these files in the iteration 

for file in os.listdir(dir):
    if file not in ignore_files: 
        with open(dir + file) as f:
            lines = f.readlines()
            
            for line in lines:
                tweet = json.loads(line.strip())
                
                if not tweet["text"].startswith("RT @"):
                    tweet_data = {}
                    tweet_data["id"] = tweet["id"]
                    tweet_data["user_id"] = tweet["from_user_id"]
                         
                    date = datetime.strptime(tweet["created_at"], "%a %b %d %H:%M:%S %z %Y")
                    tweet_data["date"] = date.date()

                    tweet_data["texts"] = tweet["text"].strip()
                    
                    tweet_data["country"] = tweet["user_location"]["country_code"]
                    if "state" in tweet["user_location"].keys():
                        tweet_data["state"] = tweet["user_location"]["state"]
                    else: 
                        tweet_data["state"] = ""
               
                    data.append(tweet_data)

In [85]:
df = pd.DataFrame(data=data)

In [86]:
df.to_csv("test.csv", index=False)